In [26]:
from PIL import Image
import numpy as np
from copy import deepcopy

Вы работаете в ФСБ в отделе К подотделе шифрования.
Спецагенты схватили трех шпионов и у каждого из них были изъяты улики.
1. у певрого была бумашка со следующим содержимым: 1f933b5
2. у второго была фотография 11.png
3. у третьего была фотография 222.bmp

После некоторых рассуждений вы решили начать с числа и для начала перевести его в десятичное представление

In [2]:
def hex_to_dec(value: str) -> str:
    return int(value,16)
    
    """
    Функция преобразует строку содержащую число в шестнадцатеричной системе счисления
    в строку содержащую число в десятичной системе счисления
    """
    #pass

In [3]:
v1 = hex_to_dec("1f933b5")
v1

33108917

После получения числа в десятичном виде и осознания что другие данные связаны с изображениями. Вы решили попытаться понять какие еще числа вы можете получить из данного и решили разложить его на простые

In [11]:
def devisors(value: int):
    d = list()
    for  i in range(2,value//2):
        if value%i==0:
            d.append(i)
            d.append(value//i)
    return d[0],d[1]
    """
    Фукнция раскладывает число на простые множители
    На вход подается только числа у которых таких множителей либо два, либо один (то есть оно уже простое).
    
    value: Натуральное число
    
    return Список двух простых чисел произведение которых дает исходное число
    
    """
    #pass


In [18]:
v2,v3 = devisors(v1)



7369

Получив два числа вы увидели странности на изображении 11.png в верхнем левом углу и решили что там шифровка. Для начала вы попробовали преобразовать цифры в двоичный код и наложить на огрези в изображении

In [14]:
def dec_to_bin(value: int):
    dv = ''
    while value > 0:
        dv += str(value % 2)
        value = value // 2
    return (dv[::-1])

     
    """
    Функция преобразует строку содержащую число из десятичной системы счисления 
    в строку содержащую число в двоичной системе счисления
    """
   # pass

In [31]:
v4 = dec_to_bin(v2)
v5 = dec_to_bin(v3)



In [ ]:
После чего вы выбрали те байты из изображения что находились на перекрестии строк и столбцов 
которые отвечали единицам двоичных чисел если их прислонить к изображению в верхний левый угол меньшее число
по вертикали а большее по горизонтали

In [38]:
filepath = "11.png"
with Image.open(filepath) as img:
    img.load()
    img = np.asarray(img)
new_img = deepcopy(img)
def translate(v4,v5):
    l = 0
    a = min(v4,v5)
    b = max(v5,v4)
    k = np.zeros((len(b),len(a)))
    for i in range(0,len(a)):
        for j in range(0,len(b)):
            if a[i] == b[j] == "1":
                k[i][j] = 1
    return k
s = translate(v4,v5).astype(int)
def translate1(img,s):
    g = []
    for i in range(len(s[0])):
        for j in range(len(s)):
            if s[i][j]==1:
                g.append(img[i][j])
    return g
h = translate1(img,s)
print(h)
 


[83, 75, 102, 102, 84, 94, 78, 32, 103, 75, 88, 87, 101, 102, 102, 88, 84, 88, 102, 102, 80, 88, 84, 102, 83, 78, 93, 75, 73, 84, 102, 102, 83, 75, 102, 102, 74, 75, 81, 103, 75, 88]


  101
  
1 101
0 000
1 101

In [42]:
def get_bytes(bin1, bin2, img):
    l = 0
    a = min(bin1,bin2)
    b = max(bin1,bin2)
    k = np.zeros((len(b),len(a)))
    for i in range(0,len(a)):
        for j in range(0,len(b)):
            if a[i] == b[j] == "1":
                k[i][j] = 1
    g = []
    for i in range(len(k[0])):
        for j in range(len(k)):
            if k[i][j]==1:
                g.append(img[i][j])
    return g

    #pass

In [43]:
byte_array = get_bytes(v4, v5, new_img)
byte_array

[83,
 75,
 102,
 102,
 84,
 94,
 78,
 32,
 103,
 75,
 88,
 87,
 101,
 102,
 102,
 88,
 84,
 88,
 102,
 102,
 80,
 88,
 84,
 102,
 83,
 78,
 93,
 75,
 73,
 84,
 102,
 102,
 83,
 75,
 102,
 102,
 74,
 75,
 81,
 103,
 75,
 88]

Далее вы посмотрев на байты изображения уловили в них отголоски кодировки ascii и решили с помощью chr преобразовать их в символы, только всем кодам кроме 32 вы решили добавить 1000 чтобы быть в области русских букв

In [49]:
def get_chars(byte_array):
    t = ''
    for i in range(len(byte_array)):
        if byte_array[i]!=32:
            l = int(byte_array[i])+1000
            t+=chr(l)
        else:
            t+=chr(byte_array[i])
    return t
get_chars(byte_array)
    #pass

In [50]:
chars = get_chars(byte_array)
chars

'лгююмцж ягрпэююрмрююирмюлжхгбмююлгюювгйягр'

Вы получили какую-то не очень длинную строчку. Возможно это ключ, но он тоже зашифрован. Вы решили попробовать самую простую кодировку - шифр Цезаря. И путем перебора расшифровали ключ

In [ ]:
def caesar_encryptor(text:str, n:int) -> str:
    """
    Функция шифрует шифром Цезаря с шагом n (n это ключ) русские предложения (пробел добавляется в конец алфавита, 
    заглавные буквы идут перед строчными: "АБВ...Яабв...я ") знаки припинания игнорируются. По достижению конца
    алфавита отсчет начинается сначала.
    Пусть например n = 2
    Шифр цезаря это когда вместо А пишется В, вместо Б пишет Г сдвиг на две буквы
    
    caesar_encryptor("Всем привет!", 1) -> "ГтёнАрсйгёу"
    """
    pass


def caesar_decryptor(text:str) -> str:
    """
    Расшифровывает Шифр Цезаря 
    caesar_decryptor(caesar_encryptor("Всем привет!", 1), 1) -> "Всем привет"
    """
    pass

def caesar_brootforce(test:str):
    """
    Перебор смещений пока не получится вменяемая строка
    """

In [ ]:
key1 = caesar_brootforce(chars)

In [ ]:
после этого вы обратили свой взор на второе изображение. слишком оно обычное и не привлекает внимание. 
возможно в нем спрятано сообщение
загрузив его вы решили посмотреть на четности чисел каждого пикселя и увидели что они совпадают всегда во всех трех каналах.
а это означает что надо вытащить четности каждого пикселя как бинарный код

In [ ]:
filepath = "222.bmp"
with Image.open(filepath) as img:
    img.load()
img = np.asarray(img)
new_img = deepcopy(img)

In [ ]:
def get_bin_array(img):
    pass

bin_array = get_bin_array(new_img)

Далее вы разбили двоичный код по 8 бит и каждый преобразовали в десятичное число а затем и в букву аналогично тому как сделали это раньше (32 пробел к остальным надо добавить 1000 и взять chr)

In [ ]:
def get_char_from_bin_array(bin_array):
    pass


char_array = get_char_from_bin_array(bin_array)

сообщение оказалось длинным и зашифрованным. но у вас есть ключ. наудачу вы попробовали расшифровать код шифрова Вижинера

In [ ]:
def vigenere_encryptor(text: str, key:str) -> str:
    """
    """
    pass


def vigenere_decryptor(text: str, key:str) -> str:
    """
    """
    pass

In [ ]:
text = vigenere_decryptor(char_array, key1)

In [4]:
import string

In [1]:
s = "абвгдеёжзийклмнопрстуфхцчшюъыьэюя "
len(s)

34